In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv("stats (26) - Copy.csv")

df.head()



,Name,player_id,year,age,ab,pa,H,1B,2B,3B,HR,K,BB,RBI,CS,SB,GIDP
0,"Pujols, Albert",405395,2020,40,152,163,34,20,8,0,6,25,9,25,0,0,4
1,"Cabrera, Miguel",408234,2020,37,204,231,51,37,4,0,10,51,24,35,0,1,3
2,"Choo, Shin-Soo",425783,2020,37,110,127,26,18,3,0,5,33,13,15,2,6,1
3,"Molina, Yadier",425877,2020,37,145,156,38,32,2,0,4,21,6,16,0,0,7
4,"Canó, Robinson",429664,2020,37,171,182,54,35,9,0,10,24,9,30,0,0,7


In [2]:
df.info()
df[['age', 'year']].describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2619 entries, 0 to 2618
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Name       2619 non-null   object
 1   player_id  2619 non-null   int64 
 2   year       2619 non-null   int64 
 3   age        2619 non-null   int64 
 4   ab         2619 non-null   int64 
 5   pa         2619 non-null   int64 
 6   H          2619 non-null   int64 
 7   1B         2619 non-null   int64 
 8   2B         2619 non-null   int64 
 9   3B         2619 non-null   int64 
 10  HR         2619 non-null   int64 
 11  K          2619 non-null   int64 
 12  BB         2619 non-null   int64 
 13  RBI        2619 non-null   int64 
 14  CS         2619 non-null   int64 
 15  SB         2619 non-null   int64 
 16  GIDP       2619 non-null   int64 
dtypes: int64(16), object(1)
memory usage: 348.0+ KB


,age,year
count,2619.000000,2619.000000
mean,28.029019,2022.638030
std,3.704714,1.642965
min,20.000000,2020.000000
25%,25.000000,2021.000000
50%,28.000000,2023.000000
75%,30.000000,2024.000000
max,42.000000,2025.000000


In [3]:
counting_stats = [
    'pa',
    'ab',
    'H',
    'HR',
    'RBI',
    'BB',
    'K'
]

age_avg = (
    df
    .groupby('age')[counting_stats]
    .mean()
    .reset_index()
    .sort_values('age')
)

age_avg.head(10)


,age,pa,ab,H,HR,RBI,BB,K
0,20,253.833333,234.500000,59.666667,7.500000,31.666667,16.666667,54.000000
1,21,304.800000,276.628571,70.514286,10.257143,35.485714,23.428571,69.800000
2,22,340.853333,306.066667,76.466667,11.026667,38.666667,28.040000,80.200000
3,23,323.510345,290.800000,71.393103,9.793103,35.296552,26.413793,75.558621
4,24,342.780612,308.918367,76.760204,10.505102,38.755102,26.892857,80.352041
5,25,336.124031,301.806202,74.662791,10.275194,38.244186,27.333333,76.624031
6,26,341.669014,306.320423,76.559859,10.320423,38.633803,28.295775,75.359155
7,27,363.011029,324.750000,81.253676,11.338235,41.272059,30.702206,79.242647
8,28,353.496183,314.522901,77.061069,11.286260,41.576336,31.309160,78.576336
9,29,362.683258,323.642534,80.484163,12.167421,42.832579,31.751131,80.665158


In [4]:
age_deltas = age_avg.copy()

for stat in counting_stats:
    age_deltas[f'{stat}_delta'] = age_deltas[stat].diff()

age_deltas = age_deltas.dropna()
age_deltas = age_deltas[['age'] + [f'{s}_delta' for s in counting_stats]]


In [5]:
age_avg_smooth = age_avg.copy()

for stat in counting_stats:
    age_avg_smooth[f'{stat}_smooth'] = (
        age_avg_smooth[stat]
        .rolling(window=3, center=True, min_periods=1)
        .mean()
    )


In [6]:
df = df.merge(age_deltas, on='age', how='left')


In [7]:
df = df.sort_values(['player_id', 'year'])

df['HR_next'] = df.groupby('player_id')['HR'].shift(-1)

reg_df = df.dropna(subset=['HR_next', 'HR', 'age'])
from sklearn.linear_model import LinearRegression

X = reg_df[['HR', 'age']]
y = reg_df['HR_next']

model = LinearRegression()
model.fit(X, y)

model.coef_, model.intercept_


(array([ 0.59847914, -0.32013874]), np.float64(14.818986355176438))

Projection models

In [1]:

import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score

import ipywidgets as widgets
from IPython.display import display, clear_output


WINDOW_N_FOR_LEADERBOARD = 50          #default number of players to show
MIN_PA_FILTER_DEFAULT = 0              #set e.g. 200 if you want to filter low PT seasons
USE_TOB_INSTEAD_OF_OBP = False         #Fantasy: OBP points = OBP decimal (False) OR OBP*PA (True)

# ---------------------------
#Helpers
# ---------------------------
def to_scalar(x):
    if isinstance(x, pd.Series):
        x = x.iloc[0]
    x = pd.to_numeric(x, errors='coerce')
    return float(x) if pd.notna(x) else np.nan

def safe_div(num, den):
    num = pd.to_numeric(num, errors='coerce')
    den = pd.to_numeric(den, errors='coerce')
    return np.where((den > 0) & np.isfinite(den) & np.isfinite(num), num / den, np.nan)

def build_X_pred(model, stat_name, current, age_now, pa_val, age_delta):
    #Ensures exact feature name ORDER matches what sklearn used during fit
    cols = list(model.feature_names_in_)
    row = {}
    for c in cols:
        if c == stat_name:
            row[c] = current
        elif c == 'age':
            row[c] = age_now
        elif c == 'pa':
            row[c] = pa_val
        elif c == f'{stat_name}_age_delta':
            row[c] = age_delta
        else:
            row[c] = np.nan
    return pd.DataFrame([row], columns=cols)


#1) Load + clean

df = pd.read_csv("stats (26) - Copy.csv").copy()
df = df.loc[:, ~df.columns.duplicated()].copy()

required_cols = {'Name','player_id','year','age','pa','ab'}
missing = required_cols - set(df.columns)
if missing:
    raise ValueError(f"Missing required columns in CSV: {missing}")

for c in ['year','age','pa','ab','player_id']:
    df[c] = pd.to_numeric(df[c], errors='coerce')

df = df.dropna(subset=['player_id','year','age']).copy()
df['year'] = df['year'].astype(int)
df['age'] = df['age'].astype(int)
df['player_id'] = df['player_id'].astype(int)

#Counting stats = everything besides identifiers
id_cols = ['Name','player_id','year','age']
counting_stats = [c for c in df.columns if c not in id_cols]

for c in counting_stats:
    df[c] = pd.to_numeric(df[c], errors='coerce')

df = df.sort_values(['player_id','year']).reset_index(drop=True)


#2) Add AB-based rate stats

rate_stats = []
if 'BB' in df.columns:
    df['BB_per_AB'] = safe_div(df['BB'], df['ab'])
    rate_stats.append('BB_per_AB')
if 'HR' in df.columns:
    df['HR_per_AB'] = safe_div(df['HR'], df['ab'])
    rate_stats.append('HR_per_AB')

#Optional extras (comment out if you don't want them)
if 'H' in df.columns:
    df['H_per_AB'] = safe_div(df['H'], df['ab'])
    rate_stats.append('H_per_AB')
if 'K' in df.columns:
    df['K_per_AB'] = safe_div(df['K'], df['ab'])
    rate_stats.append('K_per_AB')

for c in rate_stats:
    df[c] = pd.to_numeric(df[c], errors='coerce')


model_stats = counting_stats + rate_stats


#3) Age averages + deltas (league-wide)

age_avg = (
    df.groupby('age')[model_stats]
      .mean()
      .reset_index()
      .sort_values('age')
)

age_deltas = age_avg.copy()
for s in model_stats:
    age_deltas[f'{s}_age_delta'] = age_deltas[s].diff()

age_deltas = age_deltas[['age'] + [f'{s}_age_delta' for s in model_stats]].dropna()
next_age_deltas = age_deltas.rename(columns={'age':'next_age'}).copy()


#4) Next-year targets for training

for s in model_stats:
    df[f'{s}_next'] = df.groupby('player_id')[s].shift(-1)

train_df = df.dropna(subset=[f'{s}_next' for s in model_stats]).copy()


#5) Train per-stat LinearRegression models
#Features: current stat, age, pa, delta at next age
#Target: next-year stat

models = {}
metrics_rows = []

for s in model_stats:
    tmp = train_df[['player_id','year','age','pa',s,f'{s}_next']].copy()
    tmp['next_age'] = tmp['age'] + 1

    tmp = tmp.merge(
        next_age_deltas[['next_age', f'{s}_age_delta']],
        on='next_age',
        how='left'
    ).dropna(subset=[f'{s}_age_delta'])

    if len(tmp) < 50:
        continue

    feature_cols = [s, 'age', 'pa', f'{s}_age_delta']
    X = tmp[feature_cols]
    y = tmp[f'{s}_next']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    m = LinearRegression()
    m.fit(X_train, y_train)
    models[s] = m

    y_pred = m.predict(X_test)
    metrics_rows.append({
        'stat': s,
        'MAE': mean_absolute_error(y_test, y_pred),
        'R2': r2_score(y_test, y_pred),
        'n_train': len(X_train),
        'n_test': len(X_test),
    })

model_metrics = pd.DataFrame(metrics_rows).sort_values('R2', ascending=False)


#6) Projection functions

def player_history(pid: int):
    cols = ['year','age','pa','ab'] + [c for c in counting_stats if c not in ['pa','ab']] + rate_stats
    cols = [c for c in cols if c in df.columns]
    return df.loc[df['player_id']==pid, ['Name'] + cols].sort_values('year')

def project_next_year_dict(pid: int):
    """
    Returns:
      meta dict, hist df, projections df (stat-by-stat), and a dict of blend projections
    """
    hist = player_history(pid)
    if hist.empty:
        return {}, hist, pd.DataFrame(), {}

    last = hist.sort_values('year').iloc[-1]
    age_now = int(last['age'])
    age_next = age_now + 1
    pa_val = to_scalar(last['pa'])

    proj_rows = []
    proj_blend = {}

    for s in model_stats:
        if s not in last.index:
            continue

        current = to_scalar(last[s])
        delta_series = age_deltas.loc[age_deltas['age']==age_next, f'{s}_age_delta']
        age_delta = to_scalar(delta_series) if len(delta_series) else np.nan

        age_only = current + age_delta if np.isfinite(age_delta) else np.nan

        reg = np.nan
        if (s in models) and np.isfinite(current) and np.isfinite(pa_val) and np.isfinite(age_delta):
            X_pred = build_X_pred(models[s], s, current, age_now, pa_val, age_delta)
            reg = float(models[s].predict(X_pred)[0])

        if np.isfinite(age_only) and np.isfinite(reg):
            blend = 0.5*age_only + 0.5*reg
        elif np.isfinite(reg):
            blend = reg
        else:
            blend = age_only

        proj_rows.append({
            'stat': s,
            'current': current,
            'age_delta(next_age)': age_delta,
            'age_only_proj': age_only,
            'regression_proj': reg,
            'blend': blend
        })
        proj_blend[s] = blend

    proj_df = pd.DataFrame(proj_rows).sort_values('stat')

    meta = {
        'Name': last['Name'],
        'player_id': pid,
        'last_year': int(last['year']),
        'age_now': age_now,
        'age_next': age_next,
        'pa_last': pa_val
    }
    return meta, hist, proj_df, proj_blend

# ---------- Optional Fantasy Points ----------
def compute_tb_from_proj(p):
    """
    TB = 1B + 2*2B + 3*3B + 4*HR
    Uses 1B if present; else infers 1B = H - 2B - 3B - HR (if possible).
    """
    hr = p.get('HR', np.nan)
    d2 = p.get('2B', np.nan)
    d3 = p.get('3B', np.nan)

    if '1B' in p and np.isfinite(p['1B']):
        s1 = p['1B']
    elif all(k in p for k in ['H','2B','3B','HR']) and all(np.isfinite([p.get('H',np.nan), d2, d3, hr])):
        s1 = p['H'] - d2 - d3 - hr
    else:
        return np.nan

    if not all(np.isfinite([s1, d2, d3, hr])):
        return np.nan
    return s1 + 2*d2 + 3*d3 + 4*hr

def fantasy_points(p):
    """
    Points = R + TB + OBP + HR + RBI + SB, each worth 1 "point"
    OBP ambiguity:
      - default: OBP decimal (0.370 adds 0.37 points)
      - if USE_TOB_INSTEAD_OF_OBP: OBP * PA (scales with opportunities)
    """
    needed_any = ['R','HR','RBI','SB']
    if not all(k in p for k in needed_any):
        return np.nan
    R, HR, RBI, SB = p.get('R',np.nan), p.get('HR',np.nan), p.get('RBI',np.nan), p.get('SB',np.nan)
    TB = compute_tb_from_proj(p)
    OBP = p.get('OBP', np.nan)

    if USE_TOB_INSTEAD_OF_OBP:
        PA = p.get('pa', np.nan)
        OBP_component = OBP * PA if np.isfinite(OBP) and np.isfinite(PA) else np.nan
    else:
        OBP_component = OBP

    parts = [R, TB, OBP_component, HR, RBI, SB]
    return sum(parts) if all(np.isfinite(x) for x in parts) else np.nan


#7) Build ONE projection table for ALL players (for leaderboard)

def build_all_projections(min_pa_last=0):
    # last season row per player
    last_rows = (
        df.sort_values(['player_id','year'])
          .groupby('player_id', as_index=False)
          .tail(1)
          .copy()
    )
    #optional PT filter on last season
    if 'pa' in last_rows.columns:
        last_rows = last_rows[last_rows['pa'] >= min_pa_last].copy()

    out = []
    for _, r in last_rows.iterrows():
        pid = int(r['player_id'])
        meta, _, _, p = project_next_year_dict(pid)
        if not meta:
            continue

        row = {
            'Name': meta['Name'],
            'player_id': pid,
            'last_year': meta['last_year'],
            'age_next': meta['age_next'],
            'pa_last': meta['pa_last']
        }

        #store projected blends as columns
        for s, val in p.items():
            row[f'proj_{s}'] = val

        #optional fantasy points if columns exist
        fp = fantasy_points(p)
        if np.isfinite(fp):
            row['proj_FantasyPoints'] = fp

        out.append(row)

    all_proj = pd.DataFrame(out)

    #Make sure numeric columns are numeric
    for c in all_proj.columns:
        if c.startswith('proj_') or c in ['pa_last','age_next','last_year']:
            all_proj[c] = pd.to_numeric(all_proj[c], errors='coerce')

    return all_proj

#Build once (cached)
ALL_PROJ = build_all_projections(min_pa_last=MIN_PA_FILTER_DEFAULT)


# 8)UI: Player lookup + Leaderboard (Excel-like sorting)

#Mode selector
mode = widgets.ToggleButtons(
    options=['Player Lookup', 'Leaderboard'],
    value='Player Lookup',
    description='Mode:',
    button_style=''
)

#Player widgets
name_box = widgets.Text(
    value="",
    placeholder="Type full/partial name (e.g., 'betts' or 'Mookie')",
    description="Search:",
    layout=widgets.Layout(width='600px')
)
player_dd = widgets.Dropdown(
    options=[],
    description="Matches:",
    layout=widgets.Layout(width='600px')
)

#Leaderboard widgets
proj_cols = [c for c in ALL_PROJ.columns if c.startswith('proj_')]
stat_dd = widgets.Dropdown(
    options=sorted(proj_cols),
    description="Sort by:",
    layout=widgets.Layout(width='600px')
)
order_dd = widgets.Dropdown(
    options=[('Highest', 'desc'), ('Lowest', 'asc')],
    value='desc',
    description="Order:",
    layout=widgets.Layout(width='250px')
)
n_slider = widgets.IntSlider(
    value=WINDOW_N_FOR_LEADERBOARD,
    min=5, max=200, step=5,
    description="Top N:",
    layout=widgets.Layout(width='400px')
)
min_pa_box = widgets.IntText(
    value=MIN_PA_FILTER_DEFAULT,
    description="Min PA (last):",
    layout=widgets.Layout(width='250px')
)
rebuild_btn = widgets.Button(
    description="Rebuild Leaderboard",
    button_style='info'
)

out = widgets.Output()

def get_matches(q, limit=25):
    q = (q or "").strip().lower()
    if not q:
        return []
    matches = (
        df[['Name','player_id']]
        .drop_duplicates()
        .assign(_n=lambda x: x['Name'].str.lower())
        .loc[lambda x: x['_n'].str.contains(q, na=False)]
        .drop(columns=['_n'])
        .head(limit)
    )
    return [(f"{row['Name']} (id={int(row['player_id'])})", int(row['player_id'])) for _, row in matches.iterrows()]

def refresh_player_matches(change=None):
    opts = get_matches(name_box.value)
    player_dd.options = opts
    if opts:
        player_dd.value = opts[0][1]

def render():
    with out:
        clear_output(wait=True)

        if mode.value == 'Player Lookup':
            if player_dd.value is None:
                print("Type a name to search.")
                return
            pid = int(player_dd.value)
            meta, hist, proj_df, proj_dict = project_next_year_dict(pid)

            print(meta)
            print("\n--- Last Year Snapshot ---")
            display(hist.tail(1))

            print("\n--- Full Prior Years ---")
            display(hist)

            print("\n--- Next Year Projections (blend) ---")
            display(proj_df[['stat','blend','age_only_proj','regression_proj']].sort_values('stat'))

            # show fantasy points if computable
            fp = fantasy_points(proj_dict)
            if np.isfinite(fp):
                print("\n--- Projected Fantasy Points ---")
                display(pd.DataFrame([{'Name': meta['Name'], 'FantasyPoints_next': fp}]))

            print("\n--- Model quality (higher R2 better) ---")
            display(model_metrics)

        else:
            #Leaderboard mode
            col = stat_dd.value
            ascending = (order_dd.value == 'asc')
            n = int(n_slider.value)

            if col not in ALL_PROJ.columns:
                print("Selected stat not found.")
                return

            board = ALL_PROJ[['Name','player_id','last_year','age_next','pa_last', col]].copy()
            board = board.dropna(subset=[col]).sort_values(col, ascending=ascending).head(n)

            print(f"Leaderboard sorted by: {col} ({'Lowest' if ascending else 'Highest'})")
            display(board)

def on_mode_change(change=None):
    render()

def on_player_change(change=None):
    if mode.value == 'Player Lookup':
        render()

def on_leaderboard_change(change=None):
    if mode.value == 'Leaderboard':
        render()

def rebuild_leaderboard(btn=None):
    global ALL_PROJ
    with out:
        clear_output(wait=True)
        print("Rebuilding leaderboard table with Min PA filter...")
    ALL_PROJ = build_all_projections(min_pa_last=int(min_pa_box.value))
    #refresh available projection columns (in case filtering changes nothing, but safe)
    new_proj_cols = [c for c in ALL_PROJ.columns if c.startswith('proj_')]
    stat_dd.options = sorted(new_proj_cols)
    render()

#wire callbacks
name_box.observe(refresh_player_matches, names='value')
player_dd.observe(on_player_change, names='value')

mode.observe(on_mode_change, names='value')
stat_dd.observe(on_leaderboard_change, names='value')
order_dd.observe(on_leaderboard_change, names='value')
n_slider.observe(on_leaderboard_change, names='value')

rebuild_btn.on_click(rebuild_leaderboard)

#initial display
controls_player = widgets.VBox([name_box, player_dd])
controls_leader = widgets.VBox([widgets.HBox([stat_dd]),
                                widgets.HBox([order_dd, n_slider]),
                                widgets.HBox([min_pa_box, rebuild_btn])])

ui = widgets.VBox([mode, controls_player, controls_leader, out])
display(ui)

#start in player mode; hide leaderboard controls until needed (simple, no CSS)
def toggle_visibility():
    if mode.value == 'Player Lookup':
        controls_player.layout.display = 'block'
        controls_leader.layout.display = 'none'
    else:
        controls_player.layout.display = 'none'
        controls_leader.layout.display = 'block'

toggle_visibility()
def _vis(change=None):
    toggle_visibility()
    render()
mode.observe(_vis, names='value')

print("✅ Ready. Use Mode to switch between Player Lookup and Leaderboard sorting.")
print("Tip: In Leaderboard mode, pick any 'proj_*' stat and choose Highest/Lowest like Excel.")


✅ Ready. Use Mode to switch between Player Lookup and Leaderboard sorting.
Tip: In Leaderboard mode, pick any 'proj_*' stat and choose Highest/Lowest like Excel.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,r2_score
import ipywidgets as widgets
from IPython.display import display,clear_output

WINDOW_N_FOR_LEADERBOARD=50
MIN_PA_FILTER_DEFAULT=0

def to_scalar(x):
    if isinstance(x,pd.Series): x=x.iloc[0]
    x=pd.to_numeric(x,errors='coerce')
    return float(x) if pd.notna(x) else np.nan

def safe_div(num,den):
    num=pd.to_numeric(num,errors='coerce')
    den=pd.to_numeric(den,errors='coerce')
    return np.where((den>0)&np.isfinite(den)&np.isfinite(num),num/den,np.nan)

def build_X_pred(model,stat_name,current,age_now,pa_val,age_delta):
    cols=list(model.feature_names_in_)
    row={}
    for c in cols:
        if c==stat_name: row[c]=current
        elif c=='age': row[c]=age_now
        elif c=='pa': row[c]=pa_val
        elif c==f'{stat_name}_age_delta': row[c]=age_delta
        else: row[c]=np.nan
    return pd.DataFrame([row],columns=cols)

def add_batting_rates(df_in):
    d=df_in.copy()
    if '1B' not in d.columns and all(c in d.columns for c in ['H','2B','3B','HR']):
        d['1B']=d['H']-d['2B']-d['3B']-d['HR']
    if all(c in d.columns for c in ['1B','2B','3B','HR']):
        d['TB']=d['1B']+2*d['2B']+3*d['3B']+4*d['HR']
    else:
        d['TB']=np.nan
    d['BA']=np.where(d['ab']>0,d['H']/d['ab'],np.nan)
    HBP=d['HBP'] if 'HBP' in d.columns else 0
    SF=d['SF'] if 'SF' in d.columns else 0
    denom=d['ab']+d['BB']+HBP+SF
    d['OBP']=np.where(denom>0,(d['H']+d['BB']+HBP)/denom,np.nan)
    d['SLG']=np.where(d['ab']>0,d['TB']/d['ab'],np.nan)
    d['OPS']=d['OBP']+d['SLG']
    for col in ['BA','OBP','SLG','OPS']:
        if col in d.columns:
            d[col]=pd.to_numeric(d[col],errors='coerce').round(4)
    if 'TB' in d.columns:
        d['TB']=pd.to_numeric(d['TB'],errors='coerce').round(0)
    return d

df=pd.read_csv("stats (26) - Copy.csv").copy()
df=df.loc[:,~df.columns.duplicated()].copy()

required_cols={'Name','player_id','year','age','pa','ab'}
missing=required_cols-set(df.columns)
if missing: raise ValueError(f"Missing required columns: {missing}")

for c in['year','age','pa','ab','player_id']:
    df[c]=pd.to_numeric(df[c],errors='coerce')

df=df.dropna(subset=['player_id','year','age']).copy()
df['year']=df['year'].astype(int)
df['age']=df['age'].astype(int)
df['player_id']=df['player_id'].astype(int)

id_cols=['Name','player_id','year','age']
counting_stats=[c for c in df.columns if c not in id_cols]
for c in counting_stats:
    df[c]=pd.to_numeric(df[c],errors='coerce')

df=add_batting_rates(df)
df=df.sort_values(['player_id','year']).reset_index(drop=True)

rate_stats=[]
if'BB'in df.columns:
    df['BB_per_AB']=safe_div(df['BB'],df['ab'])
    rate_stats.append('BB_per_AB')
if'HR'in df.columns:
    df['HR_per_AB']=safe_div(df['HR'],df['ab'])
    rate_stats.append('HR_per_AB')
if'H'in df.columns:
    df['H_per_AB']=safe_div(df['H'],df['ab'])
    rate_stats.append('H_per_AB')
if'K'in df.columns:
    df['K_per_AB']=safe_div(df['K'],df['ab'])
    rate_stats.append('K_per_AB')

model_stats=[c for c in counting_stats+rate_stats if c in df.columns]

age_avg=df.groupby('age')[model_stats].mean().reset_index().sort_values('age')
age_deltas=age_avg.copy()
for s in model_stats:
    age_deltas[f'{s}_age_delta']=age_deltas[s].diff()
age_deltas=age_deltas[['age']+[f'{s}_age_delta'for s in model_stats]].dropna()
next_age_deltas=age_deltas.rename(columns={'age':'next_age'})

for s in model_stats:
    df[f'{s}_next']=df.groupby('player_id')[s].shift(-1)

train_df=df.dropna(subset=[f'{s}_next'for s in model_stats]).copy()

models={}
metrics_rows=[]
for s in model_stats:
    tmp=train_df[['player_id','year','age','pa',s,f'{s}_next']].copy()
    tmp['next_age']=tmp['age']+1
    tmp=tmp.merge(next_age_deltas[['next_age',f'{s}_age_delta']],on='next_age',how='left').dropna(subset=[f'{s}_age_delta'])
    if len(tmp)<50: 
        continue
    X=tmp[[s,'age','pa',f'{s}_age_delta']]
    y=tmp[f'{s}_next']
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
    m=LinearRegression()
    m.fit(X_train,y_train)
    models[s]=m
    y_pred=m.predict(X_test)
    metrics_rows.append({'stat':s,'MAE':mean_absolute_error(y_test,y_pred),'R2':r2_score(y_test,y_pred)})

model_metrics=pd.DataFrame(metrics_rows).sort_values('R2',ascending=False)

def project_next_year(pid:int):
    hist=df[df['player_id']==pid].sort_values('year').copy()
    if hist.empty: 
        return{},hist,pd.DataFrame(),pd.DataFrame()
    last=hist.iloc[-1]
    age_now=int(last['age']);age_next=age_now+1
    pa_val=to_scalar(last['pa'])
    proj={}
    rows=[]
    for s in model_stats:
        current=to_scalar(last.get(s,np.nan))
        delta_series=age_deltas.loc[age_deltas['age']==age_next,f'{s}_age_delta']
        age_delta=to_scalar(delta_series) if len(delta_series) else np.nan
        age_only=current+age_delta if np.isfinite(current) and np.isfinite(age_delta) else np.nan
        reg=np.nan
        if(s in models) and np.isfinite(current) and np.isfinite(pa_val) and np.isfinite(age_delta):
            X_pred=build_X_pred(models[s],s,current,age_now,pa_val,age_delta)
            reg=float(models[s].predict(X_pred)[0])
        blend=0.5*age_only+0.5*reg if np.isfinite(age_only) and np.isfinite(reg) else np.nan
        proj[s]=blend
        rows.append({'stat':s,'blend':blend,'age_only_proj':age_only,'regression_proj':reg})
    meta={'Name':last['Name'],'player_id':int(last['player_id']),'last_year':int(last['year']),'age_now':age_now,'age_next':age_next,'pa_last':pa_val}
    proj_table=pd.DataFrame(rows).sort_values('stat')

    proj_wide={'Name':meta['Name'],'player_id':meta['player_id'],'last_year':meta['last_year'],'age_next':meta['age_next']}
    for k,v in proj.items():
        proj_wide[f'proj_{k}']=v

    proj_rates=add_batting_rates(pd.DataFrame([{
        'ab':proj.get('ab',np.nan),
        'H':proj.get('H',np.nan),
        'BB':proj.get('BB',np.nan),
        '2B':proj.get('2B',np.nan),
        '3B':proj.get('3B',np.nan),
        'HR':proj.get('HR',np.nan),
        '1B':proj.get('1B',np.nan),
        'HBP':proj.get('HBP',0),
        'SF':proj.get('SF',0)
    }]))

    proj_wide['proj_BA']=round(to_scalar(proj_rates['BA']),4)
    proj_wide['proj_OBP']=round(to_scalar(proj_rates['OBP']),4)
    proj_wide['proj_SLG']=round(to_scalar(proj_rates['SLG']),4)
    proj_wide['proj_OPS']=round(to_scalar(proj_rates['OPS']),4)
    proj_wide['proj_TB']=round(to_scalar(proj_rates['TB']),0)

    for col in list(proj_wide.keys()):
        if col.startswith('proj_') and col not in ['proj_TB']:
            proj_wide[col]=to_scalar(proj_wide[col])
            if np.isfinite(proj_wide[col]): proj_wide[col]=round(proj_wide[col],4)

    proj_wide_df=pd.DataFrame([proj_wide])
    return meta,hist,proj_table,proj_wide_df

def build_all_projections(min_pa_last=0):
    last_rows=df.sort_values(['player_id','year']).groupby('player_id').tail(1)
    last_rows=last_rows[last_rows['pa']>=min_pa_last]
    out=[]
    for _,r in last_rows.iterrows():
        meta,_,_,proj_wide_df=project_next_year(int(r['player_id']))
        if proj_wide_df.empty: 
            continue
        out.append(proj_wide_df.iloc[0].to_dict())
    return pd.DataFrame(out)

ALL_PROJ=build_all_projections(MIN_PA_FILTER_DEFAULT)
proj_cols=sorted([c for c in ALL_PROJ.columns if c.startswith('proj_')])

mode=widgets.ToggleButtons(options=['Player Lookup','Leaderboard'],value='Player Lookup',description='Mode:')
name_box=widgets.Text(value="",placeholder="Type full/partial name",description="Search:",layout=widgets.Layout(width='600px'))
player_dd=widgets.Dropdown(options=[],description="Matches:",layout=widgets.Layout(width='600px'))

stat_dd=widgets.Dropdown(options=proj_cols,description="Sort by:",layout=widgets.Layout(width='600px'))
order_dd=widgets.Dropdown(options=[('Highest','desc'),('Lowest','asc')],value='desc',description="Order:")
n_slider=widgets.IntSlider(value=WINDOW_N_FOR_LEADERBOARD,min=5,max=200,step=5,description="Show N:")
out=widgets.Output()

def get_matches(q,limit=25):
    q=(q or"").strip().lower()
    if not q: return[]
    matches=df[['Name','player_id']].drop_duplicates()
    matches=matches[matches['Name'].str.lower().str.contains(q,na=False)].head(limit)
    return[(f"{row['Name']} (id={int(row['player_id'])})",int(row['player_id']))for _,row in matches.iterrows()]

def refresh_player_matches(change=None):
    opts=get_matches(name_box.value)
    player_dd.options=opts
    if opts: player_dd.value=opts[0][1]

def clean_hist_for_display(hist):
    base=['Name','player_id','year','age','pa','ab']
    stats=['H','1B','2B','3B','HR','BB','K','RBI','SB','CS','GIDP']
    rates=['BA','OBP','SLG','OPS','TB','BB_per_AB','HR_per_AB','H_per_AB','K_per_AB']
    keep=[c for c in base+stats+rates if c in hist.columns]
    out_hist=hist[keep].sort_values('year').copy()
    for c in ['BA','OBP','SLG','OPS','BB_per_AB','HR_per_AB','H_per_AB','K_per_AB']:
        if c in out_hist.columns:
            out_hist[c]=pd.to_numeric(out_hist[c],errors='coerce').round(4)
    if 'TB' in out_hist.columns:
        out_hist['TB']=pd.to_numeric(out_hist['TB'],errors='coerce').round(0)
    return out_hist

def render():
    with out:
        clear_output(wait=True)
        if mode.value=='Player Lookup':
            if player_dd.value is None:
                print("Type a name to search.")
                return
            meta,hist,proj_table,proj_wide_df=project_next_year(int(player_dd.value))
            print(meta)
            print("\n---Last Year Snapshot---")
            display(clean_hist_for_display(hist).tail(1))
            print("\n---Full Prior Years---")
            display(clean_hist_for_display(hist))
            print("\n---Next Year Projections (blend)---")
            display(proj_table[['stat','blend','age_only_proj','regression_proj']])
            print("\n---Projected BA/OBP/SLG/OPS (manual)---")
            show_cols=[c for c in ['proj_BA','proj_OBP','proj_SLG','proj_OPS'] if c in proj_wide_df.columns]
            display(proj_wide_df[['Name','player_id','age_next']+show_cols])
            print("\n---Model quality---")
            display(model_metrics)
        else:
            col=stat_dd.value
            asc=(order_dd.value=='asc')
            n=int(n_slider.value)
            board=ALL_PROJ[['Name','player_id','last_year','age_next',col]].dropna(subset=[col]).sort_values(col,ascending=asc).head(n)
            print(f"Leaderboard sorted by: {col} ({'Lowest' if asc else 'Highest'})")
            display(board)

name_box.observe(refresh_player_matches,names='value')
player_dd.observe(lambda change:render(),names='value')
mode.observe(lambda change:render(),names='value')
stat_dd.observe(lambda change:render(),names='value')
order_dd.observe(lambda change:render(),names='value')
n_slider.observe(lambda change:render(),names='value')

display(widgets.VBox([mode,name_box,player_dd,stat_dd,widgets.HBox([order_dd,n_slider]),out]))
render()
